# Setup
This section contains all necessary setup such as libraries and model creation.

In [1]:
import numpy as np

from src.data.data_helper import get_raw_data_as_dataframe

In [2]:
raw_data = get_raw_data_as_dataframe()
raw_data.head()

,time,measurement,label,source
0,0.0000,1.650881,0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
1,0.0002,1.654559,0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
2,0.0004,1.650881,0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
3,0.0006,1.654559,0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
4,0.0008,1.654559,0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...


In [3]:
raw_data.shape

(8030000, 4)

## Pre-processor Calibration
Due to the fact that all data needs to be pre-processed using the same bandpass filter and normalization the pre-processor needs to be calibrated. This is done by calculating the coefficients of the bandpass filter and the normalization parameters.

These parameters will then be passed to the selected model class.

In [4]:
from src.models.preprocessing.preprocessor import SignalPreprocessor

pre_processor = SignalPreprocessor(
    low_freq=20.0,
    high_freq=500.0,
    fs=5000.0,
    order=4
)

In [5]:
pre_processor.calibrate(raw_data)

## Data Parsing
This section deals with segmenting the data into usable segements with correct labeling. After that we pre-process the data using the pre-processor which is valibrated above. Once the data is pre-processed we apply one-hot encoding to the labels to allow for the use of F1 score. Finally we split the data into training and validation sets.

In [6]:
from src.data.data_helper import segement_data

In [7]:
segmented_data = segement_data(raw_data, window_length=200*5, overlap=50*5)

In [8]:
segmented_data.head()

,window_data,label,source
0,"[1.6508805694015458, 1.654558881124963, 1.6508...",0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
1,"[1.6435239459547115, 1.6435239459547115, 1.643...",0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
2,"[1.6361673225078774, 1.6361673225078774, 1.636...",0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
3,"[1.647202257678129, 1.647202257678129, 1.65088...",0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
4,"[1.6435239459547115, 1.6435239459547115, 1.643...",0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...


In [9]:
segmented_data.iloc[50]

window_data    [2.0628514824242603, 2.0371033003603407, 2.015...
label                                                          1
source         C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
Name: 50, dtype: object

In [10]:
num_classes = segmented_data['label'].nunique()
num_classes

4

In [11]:
import tensorflow as tf

y_data = np.array(segmented_data['label'].values)
y_data = tf.keras.utils.to_categorical(y_data, num_classes=num_classes)

# I apologize for this horrible line. Can be fixed by fixing the data_helper
X_data = np.stack(segmented_data.drop(columns=['label', 'source'])['window_data'].values)

In [12]:
X_data.shape

(10658, 1000)

In [13]:
X_data = pre_processor.batch_pre_process(X_data)
X_data.shape

(10658, 1000)

In [14]:
y_data.shape

(10658, 4)

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
        X_data, y_data, test_size=0.2
    )

In [16]:
X_train.shape

(8526, 1000)

In [17]:
X_val.shape

(2132, 1000)

## Model Setup
Import model classes and create instances of the models.

In [18]:
from src.models.LSTM.LSTM import LSTM
from src.models.LSTM_STFT.LSTM_STFT import LSTM_STFT
from src.models.LSTM_STFT_Dense.LSTM_STFT_Dense import LSTM_STFT_Dense

In [19]:
LSTM_model = LSTM(input_shape=X_data.shape[1], num_classes=num_classes)

In [20]:
LSTM_STFT_model = LSTM_STFT(input_shape=X_data.shape[1], num_classes=num_classes)

In [21]:
LSTM_STFT_Dense_model = LSTM_STFT_Dense(input_shape=X_data.shape[1], num_classes=num_classes)

# Train Model
Train the models

In [22]:
LSTM_model.train(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, epochs=10, verbose=2)

Epoch 1/10
267/267 - 4s - 14ms/step - f1_score: 0.4522 - loss: 0.4513 - val_f1_score: 0.4681 - val_loss: 0.3323
Epoch 2/10
267/267 - 1s - 3ms/step - f1_score: 0.5434 - loss: 0.3318 - val_f1_score: 0.5881 - val_loss: 0.3244
Epoch 3/10
267/267 - 1s - 3ms/step - f1_score: 0.5832 - loss: 0.3341 - val_f1_score: 0.5625 - val_loss: 0.3377
Epoch 4/10
267/267 - 1s - 4ms/step - f1_score: 0.5912 - loss: 0.3212 - val_f1_score: 0.5847 - val_loss: 0.3127
Epoch 5/10
267/267 - 1s - 5ms/step - f1_score: 0.5981 - loss: 0.3277 - val_f1_score: 0.5832 - val_loss: 0.3152
Epoch 6/10
267/267 - 1s - 5ms/step - f1_score: 0.6200 - loss: 0.3132 - val_f1_score: 0.5669 - val_loss: 0.3343
Epoch 7/10
267/267 - 2s - 6ms/step - f1_score: 0.6002 - loss: 0.3208 - val_f1_score: 0.5793 - val_loss: 0.3144
Epoch 8/10
267/267 - 1s - 4ms/step - f1_score: 0.6028 - loss: 0.3151 - val_f1_score: 0.6310 - val_loss: 0.3116
Epoch 9/10
267/267 - 1s - 4ms/step - f1_score: 0.6182 - loss: 0.3147 - val_f1_score: 0.6178 - val_loss: 0.3331


In [23]:
LSTM_STFT_model.train(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, epochs=10, verbose=2)

Epoch 1/10
267/267 - 3s - 13ms/step - f1_score: 0.4927 - loss: 0.4045 - val_f1_score: 0.5501 - val_loss: 0.3294
Epoch 2/10
267/267 - 1s - 5ms/step - f1_score: 0.5664 - loss: 0.3246 - val_f1_score: 0.5734 - val_loss: 0.3230
Epoch 3/10
267/267 - 1s - 5ms/step - f1_score: 0.5853 - loss: 0.3183 - val_f1_score: 0.5547 - val_loss: 0.3152
Epoch 4/10
267/267 - 1s - 5ms/step - f1_score: 0.5803 - loss: 0.3218 - val_f1_score: 0.5992 - val_loss: 0.3099
Epoch 5/10
267/267 - 1s - 6ms/step - f1_score: 0.6040 - loss: 0.3128 - val_f1_score: 0.6159 - val_loss: 0.3151
Epoch 6/10
267/267 - 1s - 6ms/step - f1_score: 0.5994 - loss: 0.3120 - val_f1_score: 0.5983 - val_loss: 0.3126
Epoch 7/10
267/267 - 1s - 5ms/step - f1_score: 0.6032 - loss: 0.3132 - val_f1_score: 0.5980 - val_loss: 0.3070
Epoch 8/10
267/267 - 1s - 5ms/step - f1_score: 0.5976 - loss: 0.3144 - val_f1_score: 0.5931 - val_loss: 0.3138
Epoch 9/10
267/267 - 1s - 5ms/step - f1_score: 0.5966 - loss: 0.3120 - val_f1_score: 0.6064 - val_loss: 0.3107


In [24]:
LSTM_STFT_Dense_model.train(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, epochs=10, verbose=2)

Epoch 1/10
267/267 - 4s - 15ms/step - f1_score: 0.4608 - loss: 0.4772 - val_f1_score: 0.4686 - val_loss: 0.3395
Epoch 2/10
267/267 - 1s - 5ms/step - f1_score: 0.4774 - loss: 0.3544 - val_f1_score: 0.5058 - val_loss: 0.3408
Epoch 3/10
267/267 - 1s - 5ms/step - f1_score: 0.5238 - loss: 0.3399 - val_f1_score: 0.5276 - val_loss: 0.3595
Epoch 4/10
267/267 - 1s - 5ms/step - f1_score: 0.5337 - loss: 0.3360 - val_f1_score: 0.5241 - val_loss: 0.3285
Epoch 5/10
267/267 - 2s - 6ms/step - f1_score: 0.5631 - loss: 0.3281 - val_f1_score: 0.5911 - val_loss: 0.3257
Epoch 6/10
267/267 - 1s - 5ms/step - f1_score: 0.5701 - loss: 0.3398 - val_f1_score: 0.5300 - val_loss: 0.3662
Epoch 7/10
267/267 - 1s - 5ms/step - f1_score: 0.5823 - loss: 0.3287 - val_f1_score: 0.6166 - val_loss: 0.3191
Epoch 8/10
267/267 - 2s - 6ms/step - f1_score: 0.5900 - loss: 0.3164 - val_f1_score: 0.5746 - val_loss: 0.3173
Epoch 9/10
267/267 - 1s - 5ms/step - f1_score: 0.5826 - loss: 0.3257 - val_f1_score: 0.5946 - val_loss: 0.3321
